In [2]:
import re
import requests
from bs4 import BeautifulSoup 
from time import sleep
import os
import selenium
from selenium import webdriver as wd

#-- txt파일을 넣을 폴더를 생성. --#
song_folder_path = "./song_folder"
if not os.path.exists(song_folder_path):
    os.makedirs(song_folder_path)

#-- 발라드/힙합 장르 선택 가능. --#
while(True):
    genre = input("음악 장르 입력(발라드, 힙합) : ")
    count = 0

    if genre == "발라드":
        gnrCode = "GN0100"
        break
    elif genre == "힙합":
        gnrCode = "GN0300"
        break
    else:
        print("장르 입력이 잘못 되었습니다. 다시 입력해주십시오.")

#-- 노래 txt 추출 파일명. --#
if genre == "발라드":
    file_name = "lyric_data_ballad.txt" 
elif genre == "힙합":
    file_name = "lyric_data_hiphop.txt" 

#-- 페이지별 크롤링용. --#
start_index = 1

#-- 크롬드라이버 열기. --#
# Window.
#driver = wd.Chrome('C:\chromedriver') 
# Mac - brew 로 설치된 chromedriver의 path.
path = '/usr/local/bin/chromedriver' 

#-- 크롬 드라이버 사용 --#
driver = wd.Chrome(path)
driver.maximize_window() # 크롬창 크기 최대


#-- 멜론 Data crawling. --#
for a in range(400):
    
    ## 드라이버 해당 url 접속. ##
    melon_url = f'https://www.melon.com/genre/song_list.htm?gnrCode={gnrCode}#params%5BgnrCode%5D={gnrCode}&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&params%5BsteadyYn%5D=N&po=pageObj&startIndex={start_index}'
    driver.get(melon_url)
    sleep(3) # 웹 페이지 로드를 보장 위해 3초 sleep.
    
    text = driver.page_source
    soup = BeautifulSoup(text, 'html.parser')
    
    #req = requests.get(melon_url, headers = headers)
    #soup = BeautifulSoup(req.text, "html.parser") --> selenium로 방식 변경.
    song_list = soup.select('tbody > tr')

    ## 곡 리스트 추출. ##
    for i, meta in enumerate(song_list,1):
        # 순위, 제목.
        rank = (50 * a) + i
        try:
            title = meta.select('a[href*=playSong]')[0].text 
        except:
            title = meta.select('.wrap_song_info .ellipsis')[0].text 
        title = title.strip()
        print(str(rank )+ '위. ', title)

        # 노래 데이터 Detail url html 추출.
        song_id_html = str(meta.select('a[href*=SongDetail]')) 
        matched = re.search(r"\'(\d+)\'", song_id_html)
        #if matched is None:
         #   return None
        song_id = matched.group(1)
        front_url = 'https://www.melon.com/song/detail.htm?songId=' 
        song_url = front_url + song_id

        # 가수.
        #singer_html = meta.select('a[href*=goArtistDetail]')[0].text 
        #singer_html = singer_html.strip()
        #print("가수:", singer_html) 
        
        # 가사가 있으면 추출.
        # Window.
        #headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
        # Mac.
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) like Gecko'}
        response = requests.get(song_url, headers=headers) 
        soup = BeautifulSoup(response.text, 'html.parser')
        
        try:
            lyric_html = str(soup.select('.section_lyric .wrap_lyric .lyric')[0])
            lyric_html = lyric_html.replace('\t','').replace('\r','').split('\n')
            lyric_html = ''.join(lyric_html)

            matched = re.search(r"-->(.*)<br/>", lyric_html)
            lyric = matched.group(1).strip()
            lyric = lyric.replace('<br/>', '\n')

            # 가사 앞뒤 빈칸 제거. 
            lyric_list = []
            for line in lyric.split('\n'):
                lyric_list.append(line.strip()) 
            lyric = ('\n').join(lyric_list)

        except:
            lyric = ""
        
        # 가사 출력.
        #print('(가사)',sep='\n') 
        #print(lyric)
        #print()

        # 파일이름 불가 문자제거.
        title = re.sub(r"[\/\:\*\?\"\<\>\|]", " ", title)

        # 노래 txt 추출본 저장 경로.
        txt_path = song_folder_path + "/" + file_name 

        # 인코딩 판단.
        try:
            f = open(txt_path, 'a', encoding='utf-8')
            f.write(lyric) 
            f.write('\n---------------------------------------------\n') # 노래별 분리.
            f.close()
            print("가사 추출 완료")
        except:
            print("가사 인코딩 실패")
        
        # IP 차단 방지용.
        sleep(1)
        
    start_index = start_index + 50

음악 장르 입력(발라드, 힙합) : 발라드


<ipython-input-2-a338261e2dcc>:44: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wd.Chrome(path)


1위.  내일부터 나도 웃을 것 같아 (Prod. By LA박피디)
가사 추출 완료
2위.  잠 못 드는 밤 네 생각은 나고
가사 추출 완료
3위.  Last Chapter(졸업.ver)
가사 추출 완료
4위.  하소연
가사 추출 완료
5위.  홀로 크리스마스
가사 추출 완료
6위.  Valley of the Shadow
가사 추출 완료
7위.  사랑은 타이밍
가사 추출 완료
8위.  첫 눈 오는 날
가사 추출 완료
9위.  Sugar Shower (Feat. 김유나)
가사 추출 완료
10위.  Suddenly
가사 추출 완료
11위.  공로상
가사 추출 완료
12위.  사랑의 온도
가사 추출 완료
13위.  첫눈처럼 너에게 가겠다
가사 추출 완료
14위.  미치도록 그립다
가사 추출 완료
15위.  마음이 내려 쌓이면
가사 추출 완료
16위.  다시 겨울
가사 추출 완료
17위.  bestie
가사 추출 완료
18위.  마지막그날 (feat. 연선)
가사 추출 완료
19위.  권태기 (Feat. 기연)
가사 추출 완료
20위.  이별이 아픈건, 헤어짐이 아니라 잊혀짐이다
가사 추출 완료
21위.  Invite
가사 추출 완료
22위.  여기까지
가사 추출 완료
23위.  이별에 대한 독백
가사 추출 완료
24위.  비의 노래 (Vocal 김진웅)
가사 추출 완료
25위.  우리 둘만 아는 비밀 (Feat. 김윤재)
가사 추출 완료
26위.  서른은 어른일까
가사 추출 완료
27위.  영하 (feat. 권민산)
가사 추출 완료
28위.  우리의 사랑이 끝나도
가사 추출 완료
29위.  Lonely Night
가사 추출 완료
30위.  우리
가사 추출 완료
31위.  눈이 내리던 어느 날 (feat. MORI)
가사 추출 완료
32위.  크리스마스 선물
가사 추출 완료
33위.  밤새 서로 미루다
가사 추출 완료
34위.  I Miss You (쿠키박스 X 산들)
가사 추출 완료
35위.  맴찢
가사 추출 완료
36위.  A Letter from Arizona


301위.  이별을 이기는 말
가사 추출 완료
302위.  엄마
가사 추출 완료
303위.  Don't Look At Me
가사 추출 완료
304위.  첫눈(feat.서지혜)
가사 추출 완료
305위.  시간이 흘러
가사 추출 완료
306위.  거짓말은 하지 말지 그랬어
가사 추출 완료
307위.  바다로보낸편지：Florianópolis
가사 추출 완료
308위.  덤덤하게
가사 추출 완료
309위.  너에게 나는 (To U)
가사 추출 완료
310위.  낮과 밤
가사 추출 완료
311위.  꽃 가든
가사 추출 완료
312위.  Beyond Life
가사 추출 완료
313위.  신기루 (feat. 최윤지)
가사 추출 완료
314위.  추억으로
가사 추출 완료
315위.  우리의 크리스마스 (with 조애란)
가사 추출 완료
316위.  나와 내가 마주한 시간에
가사 추출 완료
317위.  대충 만든 노래 (Feat. Sun Ahn)
가사 추출 완료
318위.  Sigh
가사 추출 완료
319위.  무궁화 꽃이 피면 (Narr. 김수현, 김진희)
가사 추출 완료
320위.  은하 (Galaxy)
가사 추출 완료
321위.  My Way (Prod. 윤민수 of VIBE)
가사 추출 완료
322위.  놀
가사 추출 완료
323위.  슬픈 사랑이야기
가사 추출 완료
324위.  진심으로 너를 사랑하고 있어 (Feat. 밈(mim))
가사 추출 완료
325위.  그날의 하루
가사 추출 완료
326위.  너를 사랑했던 날 (feat. 우이경)
가사 추출 완료
327위.  Episode
가사 추출 완료
328위.  매듭 (Feat. lrea)
가사 추출 완료
329위.  너의 빈자리
가사 추출 완료
330위.  어떤 여행
가사 추출 완료
331위.  이별 일기
가사 추출 완료
332위.  얼룩
가사 추출 완료
333위.  아무렇지 않은 너 (Vocal. 나수민)
가사 추출 완료
334위.  차가운 봄 (Vocal by 주대건)
가사 추출 완료
335위.  그 말 진심이 

608위.  밤하늘
가사 추출 완료
609위.  내 잘못이죠
가사 추출 완료
610위.  혼자 (Vocal. 김소영)
가사 추출 완료
611위.  사랑은 지울 수 없어요
가사 추출 완료
612위.  연인 (키스)
가사 추출 완료
613위.  변화
가사 추출 완료
614위.  비가 이끈 곳
가사 추출 완료
615위.  그댄 나를 빛나게 해줘요
가사 추출 완료
616위.  여전히 오늘도(feat.목연수)
가사 추출 완료
617위.  별빛 달빛 그리움
가사 추출 완료
618위.  헤어지잔 말이 그렇게 쉬웠니 (Vocal. 성희)
가사 추출 완료
619위.  글월 (Vocal. YuL)
가사 추출 완료
620위.  Dear
가사 추출 완료
621위.  꽃처럼
가사 추출 완료
622위.  육십센치 위 우주 (Feat. 정지현)
가사 추출 완료
623위.  안녕
가사 추출 완료
624위.  이 밤 (Feat. 노윤하)
가사 추출 완료
625위.  네가 떠나간 세상은 내겐 참 낯설어
가사 추출 완료
626위.  그 계절 그때에
가사 추출 완료
627위.  치맥 Song
가사 추출 완료
628위.  흘러간다
가사 추출 완료
629위.  사랑한다 그리고 사랑한다
가사 추출 완료
630위.  가끔은
가사 추출 완료
631위.  Color
가사 추출 완료
632위.  Tell me
가사 추출 완료
633위.  너이기에 (feat. 나은)
가사 추출 완료
634위.  그 여자가 나예요 (금혼령 X 이소정)
가사 추출 완료
635위.  가끔 내가 생각나는지
가사 추출 완료
636위.  숨바꼭질
가사 추출 완료
637위.  끝나지 않을 이야기로만 남아도..
가사 추출 완료
638위.  사랑 후기
가사 추출 완료
639위.  Love is you
가사 추출 완료
640위.  한강 참 예쁘다
가사 추출 완료
641위.  플래너 인생에 빛난 순간
가사 추출 완료
642위.  Estrella
가사 추출 완료
643위.  누니
가사 추출 완료
644위.  그러지 말았어야 해요
가사 추출 완료


KeyboardInterrupt: 

#### 이건 내가 selenium 및 chromedriver 실행 테스트 해보려고 적어둔 로직임!! ###

# selenium의 webdriver를 가져온다.
from selenium import webdriver
# 브라우저에 띄우고 싶은 URL 입력. 이 경우 로그인부터 하기 위해 로그인창에 접근한다.
browser.get('https://login.toast.com')
# 로그인 창에서 자동으로 정보를 입력
browser.find_element_by_xpath("//input[@type='text']").send_keys("********")
browser.find_element_by_xpath("//input[@type='password']").send_keys("********")
# 입력된 로그인 정보 제출
browser.find_element_by_xpath("//button[@type='submit']").click()
# 로그인 이후, 확인이 필요한 페이지로 이동
browser.get('https://after-login.toast.com')

# 확인하고 싶은 로직. 여기선 테스트로 refresh 를 0.5에 한번씩 하도록 했다.
import time
max_time = 5
start_time = 0
refresh_time_in_seconds = 0.5
while(start_time < max_time):
    browser.refresh()
    start_time += refresh_time_in_seconds
    time.sleep(refresh_time_in_seconds)

browser.quit()